In [1]:
# basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.cluster import KMeans

#others
from xgboost import XGBRegressor
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import time
import xarray as xr
import sherpa

# Variables from config file
from config import BASE_DIR, FILE_NAMES, LABELS, ATTRIBUTES


In [2]:
# Load the datasets
# Load the dataset
df_metadata = pd.read_excel(f"{BASE_DIR}/FilledDataset2012.xlsx", sheet_name="Header")
df_data_original = pd.read_csv(f"{BASE_DIR}/dataset.csv")

df_interp_100 = (
    pd.read_csv(f"{BASE_DIR}/dataset_interp_100.csv")
    .drop(labels=["year", "month", "name", "Observer", "NumMos", "MinYear", "MaxYear", "Status2010"], axis=1)
)

df_interp_50 = (
    pd.read_csv(f"{BASE_DIR}/dataset_interp_50.csv")
    .drop(labels=["year", "month", "name", "Observer", "NumMos", "MinYear", "MaxYear", "Status2010"], axis=1)
)

df_6grids = pd.read_csv(f"{BASE_DIR}/dataset_5girds.csv")
df_6grids = df_6grids.drop(
    labels=[
        "year", "month", "name", "Observer", "NumMos", "MinYear", "MaxYear", "Status2010",
        "air2m", "air1000_500", "hgt500", "hgt1000", "omega500",
        "pottemp1000-500", "pottemp1000-850", "pr_wtr", "shum-uwnd-700",
        "shum-uwnd-925", "shum-vwnd-700", "shum-vwnd-950", "shum700",
        "shum925", "slp"
    ],
    axis=1
)

In [36]:
np.where(df_metadata["Name"] == "Big Bog")[0]
np.where(df_metadata["Name"] == "HILO")[0]
np.where(df_metadata["Name"] == "HONOLULU")[0]
np.where(df_metadata["Name"] == "LIHUE")[0]

array([2070])

In [46]:
skn_bigbog = df_metadata[df_metadata["Name"] == "Big Bog"]['SKN'].iloc[0]
type(skn_bigbog), skn_bigbog

(numpy.float64, 276.0)

In [32]:
df_metadata["Name"]

0                   KALAE
1             MORSE FIELD
2         KALAE S TRK STA
3                  MANUKA
4       KAHUKU MAUKA 2.10
              ...        
2219             FIELD 30
2220            FIELD 7-A
2221             PUU AUAU
2222         Moloaa Dairy
2223       FLD 960-MOLOAA
Name: Name, Length: 2224, dtype: object

In [ ]:
# Hilo station on Big Island;
# Big Bog station on Maui;
# Honolulu Station on Oʻahu;
# Līhuʻe station on Kauaʻi.